# Imports

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import imageio
import numpy as np
import itertools
import torch.optim as optim
import matplotlib.pyplot as plt
import os,time

from torchvision import transforms, datasets
from torch.autograd import Variable



# Parameters

In [2]:
lr = 0.0002
img_size = 32
batch_size = 512
num_epochs = 40
save_dir = '/home/abdullah/Documents/ai/models/Own implementation of Gan/Conditional Gan/Model results'
G_input_dim = 100
G_output_dim = 1
D_input_dim = 1
D_output_dim = 1
label_dim = 10

In [3]:
def to_var(x):
    x = x.cuda()
    return Variable(x)

def denorm(x):
    out = (x+1)/2
    return out.clamp(0,1)    

In [4]:
def plot_loss(d_losses, g_losses, num_epoch, save=True ,show=False):
    fig, ax = plt.subplots()
    ax.set_xlim(0, num_epochs)
    ax.set_ylim(0, max(np.max(g_losses), np.max(d_losses))*1.1)
    plt.xlabel('Epoch {0}'.format(num_epoch + 1))
    plt.ylabel('Loss values')
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.legend()

    # save figure
    if save:
        save_fn = save_dir + 'MNIST_cDCGAN_losses_epoch_{:d}'.format(num_epoch + 1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

def plot_result(generator, noise, label, num_epoch, save=True, show=False, fig_size=(5, 5)):
    generator.eval()

    noise = Variable(noise.cuda())
    label = Variable(label.cuda())
    gen_image = generator(noise, label)
    gen_image = denorm(gen_image)

    generator.train()

    n_rows = np.sqrt(noise.size()[0]).astype(np.int32)
    n_cols = np.sqrt(noise.size()[0]).astype(np.int32)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    for ax, img in zip(axes.flatten(), gen_image):
        ax.axis('off')
        ax.set_adjustable('box-forced')
        ax.imshow(img.cpu().data.view(img_size, img_size).numpy(), cmap='gray', aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch+1)
    fig.text(0.5, 0.04, title, ha='center')

    # save figure
    if save:
        save_fn = save_dir + 'MNIST_cDCGAN_epoch_{:d}'.format(num_epoch+1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

        

# Data Loading

In [5]:
transform = transforms.Compose([
    transforms.Scale(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5,0.5,.5), std = (.5,.5,.5))
])


mnist_data = datasets.MNIST(root="/home/abdullah/Documents/ai/models/Own implementation of Gan/Untitled Folder/DCGAN/mnist",
                         train=True,
                         transform=transform,
                         download=False)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data,
                                          batch_size=batch_size,
                                          shuffle=True)



/home/abdullah/miniconda3/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


# Model Architecture

In [6]:
class generator(nn.Module):
    def __init__(self,d=128):
        
        super(generator,self).__init__()
        
        self.deconv1_z = nn.ConvTranspose2d( 100 , d*2 , 4 , 1 , 0)
        self.deconv1_y = nn.ConvTranspose2d( 10, d*2 , 4 , 1, 0)
        
        self.batch_norm_1 = nn.BatchNorm2d(d*2)
        
        self.deconv2 = nn.ConvTranspose2d(d*4 , d*2, 4, 2, 1)
        self.batch_norm_2 = nn.BatchNorm2d(d*2)
        
        self.deconv3 = nn.ConvTranspose2d(d*2 , d , 4 , 2 , 1)
        self.batch_norm_3 = nn.BatchNorm2d(d)
        
        self.deconv4 = nn.ConvTranspose2d(d , 1 , 4, 2 , 1)
        
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean , std)
            
    def forward(self , input , label):
        x = F.relu(self.batch_norm_1(self.deconv1_z(input)))
        y = F.relu(self.batch_norm_1(self.deconv1_y(label)))
        
        x = torch.cat([x,y],1)
        
        x = F.relu(self.batch_norm_2(self.deconv2(x)))
        
        x = F.relu(self.batch_norm_3(self.deconv3(x)))
        
        x = F.tanh(self.deconv4(x))
        
        return x

class discriminator(nn.Module):
    def __init__(self, d = 128):
        
        super( discriminator, self).__init__()
        print("hello")
        self.conv1_x = nn.Conv2d(1 , d//2 , 4, 2, 1)
        self.conv1_y = nn.Conv2d(10 , d//2 , 4, 2, 1)
        
        self.conv2 = nn.Conv2d( d, d*2, 4, 2, 1)
        self.batch_norm_2 = nn.BatchNorm2d(d*2)
        
        self.conv3 = nn.Conv2d( d*2, d*4, 4 , 2, 1)
        self.batch_norm_3 = nn.BatchNorm2d(d*4)
        
        self.conv4 = nn.Conv2d( d*4, 1, 4, 1, 0)
        
    def weight_init(self, mean , std):
        for m in self._modules:
            normal_init(self._modules[m], mean , std)
            
    def forward(self, input  , label):
        x = F.leaky_relu(self.conv1_x(input),0.2)
        y = F.leaky_relu(self.conv1_y(label),0.2)
        
        x = torch.cat([x,y],1)
        
        x = F.leaky_relu(self.batch_norm_2(self.conv2(x)),0.2)
        
        x = F.leaky_relu(self.batch_norm_3(self.conv3(x)),0.2)
        
        x = F.sigmoid(self.conv4(x))
        
        return x
    
    
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()       
        
        
generator = generator()
discriminator = discriminator()

generator.weight_init(mean = 0 , std = 0.02)
discriminator.weight_init(mean = 0 , std = 0.02)

generator.cuda()
discriminator.cuda()

hello


discriminator(
  (conv1_x): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv1_y): Conv2d(10, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch_norm_2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch_norm_3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
)

# Optimizations

In [7]:
opt_Gen = optim.Adam(generator.parameters(),lr = lr , betas = (.5, .999))
opt_Disc = optim.Adam(discriminator.parameters(),lr = lr , betas = (.5, .999))

criterion = torch.nn.BCELoss()

# Training 

In [11]:
# num_test_samples = 10*10
temp_noise = torch.randn(label_dim, G_input_dim)
fixed_noise = temp_noise
fixed_c = torch.zeros(label_dim, 1)
for i in range(9):
    fixed_noise = torch.cat([fixed_noise, temp_noise], 0)
    temp = torch.ones(label_dim, 1) + i
    fixed_c = torch.cat([fixed_c, temp], 0)

fixed_noise = fixed_noise.view(-1, G_input_dim, 1, 1)
fixed_label = torch.zeros(G_input_dim, label_dim)
fixed_label.scatter_(1, fixed_c.type(torch.LongTensor), 1)
fixed_label = fixed_label.view(-1, label_dim, 1, 1)

# label preprocess
onehot = torch.zeros(label_dim, label_dim)
onehot = onehot.scatter_(1, torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).view(label_dim, 1), 1).view(label_dim, label_dim, 1, 1)
fill = torch.zeros([label_dim, label_dim, img_size, img_size])
for i in range(label_dim):
    fill[i, i, :, :] = 1

D_avg_losses = []
G_avg_losses = []





for epoch in range(1, num_epochs +1):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    generator.train()
    if epoch == 5 or epoch == 10:
        opt_Gen.param_groups[0]['lr'] /=2
        opt_Disc.param_groups[0]['lr'] /=2
        
    
    for i, (real_images,real_labels) in enumerate(data_loader):
        minibatch = real_images.size()[0]
        real_images = Variable(real_images.cuda())
        
        #labels
        y_real = Variable(torch.ones(minibatch).cuda())
        y_fake = Variable(torch.zeros(minibatch).cuda())
        c_fill = Variable(fill[real_labels].cuda())
        z_ = torch.randn(minibatch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())
        ## Train Discriminator             
        # first with real data
        
        D_real_decision = discriminator(real_images, c_fill).squeeze()
        D_real_loss = criterion(D_real_decision, y_real)
        
        # Then with fake data
        
        c_ = (torch.rand(minibatch, 1) * label_dim).type(torch.LongTensor).squeeze()
        c_onehot_ = Variable(onehot[c_].cuda())
        c_fill_ = Variable(fill[c_].cuda())
        
        generator_image = generator(z_, c_onehot_) 
        D_fake_decision = discriminator(generator_image,c_fill_).squeeze()
        D_fake_loss = criterion(D_fake_decision, y_fake)
        
        # Optimization
        discriminator.zero_grad()
        D_loss = D_fake_loss + D_real_loss
        D_loss.backward()
        opt_Disc.step()
        
        # Train Generator
        
        generator_image = generator(z_, c_onehot_)
        c_fill = Variable(fill[c_].cuda())
        D_fake_decision = discriminator(generator_image,c_fill).squeeze()
        G_loss = criterion(D_fake_decision, y_real)
        
        # Optimization
        generator.zero_grad()
        G_loss.backward()
        opt_Gen.step()
        
        D_losses.append(D_loss.data[0])
        G_losses.append(G_loss.data[0])
        
        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
            % (epoch+1, num_epochs, i+1, len(data_loader), D_loss.data[0], G_loss.data[0]))
        
    torch.save(generator.state_dict(), '/home/abdullah/Documents/ai/models/Own implementation of Gan/Conditional Gan/model_weights/generator_param.pkl')
    torch.save(discriminator.state_dict(), '/home/abdullah/Documents/ai/models/Own implementation of Gan/Conditional Gan/model_weights/discriminator_param.pkl')
    
    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    plot_loss(D_avg_losses, G_avg_losses, epoch, save=True)

    # Show result for fixed noise
    plot_result(generator, fixed_noise, fixed_label, epoch, save=True)
    
        

/home/abdullah/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/abdullah/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/abdullah/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


KeyboardInterrupt: 

In [19]:
# Make gif
loss_plots = []
gen_image_plots = []
for epoch in range(2,24):
    # plot for generating gif
    save_fn1 = 'Model resultsMNIST_cDCGAN_losses_epoch_{:d}'.format(epoch + 1) + '.png'
    loss_plots.append(imageio.imread(save_fn1))

    save_fn2 = 'Model resultsMNIST_cDCGAN_epoch_{:d}'.format(epoch + 1) + '.png'
    gen_image_plots.append(imageio.imread(save_fn2))

imageio.mimsave('MNIST_cDCGAN_losses_epochs_{:d}'.format(num_epochs) + '.gif', loss_plots, fps=5)
imageio.mimsave('MNIST_cDCGAN_epochs_{:d}'.format(num_epochs) + '.gif', gen_image_plots, fps=5)